In [3]:
search_for = 125
start_from = 0
threads = 10
thread = 6
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004191160202026367
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 35
all cases: 320327931
type: [1, 1, 1, 1, 125] 125
cases of this type: 244140625
100000 17.958109299120167
200000 24.62825555837076
300000 23.698401463074397
400000 27.33372855410183
500000 28.30223236622348
600000 24.7200906626554
700000 23.23299420084043
800000 24.523258178861752
900000 25.42768385424587
1000000 11.885540998320675
1100000 21.710206722112236
1200000 23.744540592757293
1300000 24.878192667634604
1400000 23.28425010396927
1500000 27.00956754841029
1600000 24.04567676321564
1700000 25.836009651621083
1800000 23.79953215034142
1900000 24.761660390481936
2000000 7.832394658968092
2100000 29.206590560499016
2200000 30.135862926184778
2300000 28.558494182440253
2400000 29.506602493890473
2500000 28.376487220167384
2600000 28.089283315444366
2700000 28.610548806160125
2800000 27.13920823138779
2900000 26.412239928370266
3000000 11.

29500000 23.54062938159543
29600000 9.378654134846515
29700000 22.64419928973463
29800000 22.77221204450548
29900000 22.371190291737715
30000000 21.487822172770443
30100000 21.135102536173196
30200000 20.167349015005257
30300000 9.602380143041307
30400000 19.714122175913626
30500000 20.530164074995547
30600000 21.353060816238806
30700000 21.894686435525994
30800000 22.81065618912008
30900000 23.153334884868013
31000000 17.418079252403547
31100000 15.715735999555518
31200000 21.964514500463874
31300000 22.862863576361253
31400000 22.856372362613257
31500000 18.78741582563376
31600000 15.772858873420596
31700000 23.625848141146847
31800000 23.079636839115015
31900000 24.158895781008127
32000000 22.880210679696855
32100000 21.536578615146247
32200000 19.341573684297522
32300000 10.510009477390929
32400000 20.122380125058662
32500000 21.06265324969184
32600000 23.22950794069555
32700000 21.854094119413677
32800000 23.96955801425433
32900000 23.074973938036464
33000000 12.354729031723673
33

59300000 19.03260171668017
59400000 19.72688019066441
59500000 18.224984759170535
59600000 9.34288319763947
59700000 17.61697387385976
59800000 18.901150270367356
59900000 19.335664188948915
60000000 19.10770275108092
60100000 9.917972861889492
60200000 19.071732830413353
60300000 20.505593603920584
60400000 20.093795418571762
60500000 21.436334339812255
60600000 22.358151117051904
60700000 20.666432410903766
60800000 20.66282401329878
60900000 19.915999028732557
61000000 20.58591426803845
61100000 8.165879256709905
61200000 18.505052592143024
61300000 17.777264101487038
61400000 18.231694873106914
61500000 16.465609558973117
61600000 9.768237547347647
61700000 18.145112850017092
61800000 18.62359508945996
61900000 17.58973784552238
62000000 14.67311660912951
62100000 14.756659193012053
62200000 19.09706309088047
62300000 20.495242289770705
62400000 19.7522078831314
62500000 21.835162136075002
62600000 20.357304853269714
62700000 20.990239325403184
62800000 20.778552669156337
62900000 

89000000 15.276854889481305
89100000 16.939990996015933
89200000 8.924588915598028
89300000 16.386644405522574
89400000 16.811611916390763
89500000 16.564188509010403
89600000 18.07338861376076
89700000 16.86754875046394
89800000 17.257465178312565
89900000 18.347731196773715
90000000 18.09612350693375
90100000 17.66284198757123
90200000 16.743410650715077
90300000 17.5895022562436
90400000 15.078182585387184
90500000 15.763056209956734
90600000 8.47303151504972
90700000 16.228876666332226
90800000 13.644459103904234
90900000 8.319059407544072
91000000 16.196734047534523
91100000 9.741720216283174
91200000 15.581542266017813
91300000 16.22620332619566
91400000 17.035188100781028
91500000 16.11085745010729
91600000 17.13115397145553
91700000 17.313428274083762
91800000 17.450590284307772
91900000 18.000086792468025
92000000 17.836570859061013
92100000 17.88868795518466
92200000 16.637214745831912
92300000 16.696336795067435
92400000 16.403165729047107
92500000 16.329718576424956
9260000

118000000 1.632662860975312
118100000 1.5250675615013551
118200000 1.5984777205382945
118300000 1.5132329141858474
118400000 1.5763022094122967
118500000 1.514771614160059
118600000 1.5479314861901328
118700000 1.708253836676701
118800000 1.7068804802919983
118900000 1.5661794162577498
119000000 1.5059922827814531
119100000 1.692476321062827
119200000 1.566630851297741
119300000 1.5485971635993017
119400000 1.5389051923675399
119500000 1.5743453925882969
119600000 1.534781391375476
119700000 1.5334758672599795
119800000 1.62939406268215
119900000 1.5467024990324252
120000000 1.562329980219555
120100000 1.5652202224559355
120200000 1.569755192397799
120300000 1.555115835255154
120400000 1.6341223644214193
120500000 1.6354076394884627
120600000 1.6016819610484083
120700000 1.5557969802433558
120800000 1.9278369546777188
120900000 1.5658645287846742
121000000 1.5512321960475963
121100000 1.5563263717340463
121200000 1.5608924184741657
121300000 1.5553611179090108
121400000 1.8164356616258

...

204400000 8.165297154705366
204500000 9.126434905960789
204600000 8.298106846257216
204700000 8.838238581214924
204800000 3.9400911310441433
204900000 9.214837336280057
205000000 8.816607328667658
205100000 9.192688492786354
205200000 8.626177547749526
205300000 8.627709826047294
205400000 5.293696331294564
205500000 7.734216012082511
205600000 8.465658295466122
205700000 8.064585542776081
205800000 8.28663903853875
205900000 7.633963711577015
206000000 7.6562858450401805
206100000 2.9069868960335636
206200000 7.872136499036069
206300000 7.646573915518441
206400000 8.200190173281527
206500000 8.408310491601446
206600000 8.407593583330424
206700000 8.184382359795677
206800000 4.409586622788793
206900000 8.698266952053087
207000000 8.664472671658075
207100000 9.013678307158012
207200000 8.670328187898996
207300000 7.598616522553539
207400000 5.668328711218288
207500000 8.988062526503853
207600000 8.553727130136636
207700000 8.3935638113383
207800000 8.438468762767473
207900000 7.89297695

233700000 6.531500517980442
233800000 6.486599998761321
233900000 6.634175242052381
234000000 6.821875362836984
234100000 6.726721199651044
234200000 6.858608431185766
234300000 4.341374612361783
234400000 5.741818018192377
234500000 3.3543056555780537
234600000 6.627431707939408
234700000 6.614016751253273
234800000 6.78274519526854
234900000 6.8835478938058055
235000000 6.306295497467344
235100000 6.394937525672781
235200000 6.74883132738887
235300000 6.688881346215849
235400000 2.356657507330763
235500000 6.852017136528874
235600000 6.804612110802588
235700000 6.600206610559451
235800000 6.642727229550169
235900000 6.918134867088839
236000000 6.812419367337339
236100000 6.835385157576978
236200000 6.817918208763981
236300000 2.9108739101951313
236400000 4.254676611586869
236500000 5.796174806397
236600000 6.5322758832328836
236700000 6.5822193425999815
236800000 6.725930886294744
236900000 6.3964651873317555
237000000 6.22262894843393
237100000 6.255035437796057
237200000 6.48378811

262400000 2.018107393887902
262500000 2.641805025528894
262600000 1.2827272984563298
262700000 0.4972668093488766
262800000 0.4950315445028226
262900000 1.281062536157116
263000000 2.851478677438261
263100000 2.0641935328041137
263200000 2.3312786986887666
263300000 2.308029523561043
263400000 2.1249739617295984
263500000 2.4036931696215023
263600000 2.273496610096027
263700000 2.7136905166646326
263800000 2.3598842538557245
263900000 2.414482908424543
264000000 2.1604608237059595
264100000 2.0952171110934423
264200000 1.9920974886120217
264300000 2.2280102874176544
264400000 2.308452608136978
264500000 1.7471072328932642
264600000 0.8517503019046466
264700000 0.4914182715192397
264800000 0.69071752036777
264900000 1.6983592757040022
265000000 2.522304006310579
265100000 2.228833664399293
265200000 1.916292134183872
265300000 2.052495538898815
265400000 2.083743409231902
265500000 2.20707992699611
265600000 2.215839985791174
265700000 2.568623199855041
265800000 2.1424494816184225
2659

290900000 1.096901700543213
291000000 1.101846376979977
291100000 1.2132196462228297
291200000 1.2412608068894804
291300000 1.009659138596159
291400000 1.0757786651926855
291500000 1.15011762961572
291600000 1.02595958476517
291700000 0.8260501690137175
291800000 1.0135116203351047
291900000 0.4863162942373276
292000000 0.19839121458163328
292100000 0.19795256405181022
292200000 0.49770771227879324
292300000 1.1210283717777265
292400000 0.7415716561706185
292500000 1.1072918001901628
292600000 1.0215431737762974
292700000 0.8401058704274298
292800000 0.9614378902593255
292900000 1.0705648676635153
best so far: 0
type: [1, 1, 1, 25, 5] 125
cases of this type: 9765625
293000000 1.1157796113943197
293100000 0.8389302372703235
293200000 0.8238110447077552
293300000 0.9120635306934841
293400000 0.6893864981754423
293500000 0.5028185989614646
293600000 0.6627212553387556
293700000 0.5993833717439294
293800000 0.8034497474867781
293900000 0.5041055628423976
294000000 0.4334666551821987
294100

best so far: 0
type: [1, 1, 125, 1, 1] 125
cases of this type: 15625
best so far: 0
type: [1, 5, 1, 1, 25] 125
cases of this type: 1953125
317300000 0.020878136933503547
317400000 0.062086972945298746
317500000 0.08624401838091479
317600000 0.04904235864586168
317700000 0.054725093072086575
317800000 0.041218662277944886
317900000 0.0372539027941068
318000000 0.04225110997379423
318100000 0.024261204495679006
318200000 0.008611364168214137
318300000 0.00803764864049554
318400000 0.0084230269847499
318500000 0.03342045759591262
318600000 0.05060446729192734
318700000 0.03663304531921082
318800000 0.023021107470619678
318900000 0.03315822316785454
319000000 0.03779005797978309
319100000 0.03506853830853833
319200000 0.021521212333230177
best so far: 0
type: [1, 5, 1, 5, 5] 125
cases of this type: 390625
319300000 0.011613638796579838
319400000 0.007726678030860424
319500000 0.00825981711007754
319600000 0.005401225918696324
best so far: 0
type: [1, 5, 1, 25, 1] 125
cases of this type: 78